In [1]:
# importing libraries
import pandas as pd
import re
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import requests
import warnings
warnings.filterwarnings('ignore')

Q1 -- Write a python program which searches all the product under a particular product from www.amazon.in. The 
product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for 
guitars.

In [2]:
driver=webdriver.Chrome() #opening automated chrome

In [3]:
driver.get('https://www.amazon.in/') #opening amazon on automated chrome browser

In [4]:
product=driver.find_element(By.ID,"twotabsearchtextbox") #search box element

In [5]:
product.send_keys(input()) # searching for product by user input

 guitars


In [6]:
#search button click
search=driver.find_element(By.ID,"nav-search-submit-button")
search.click()

Q2-- In the above question, now scrape the following details of each product listed in first 3 pages of your search 
results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then 
scrape all the products available under that product name. Details to be scraped are: "Brand  
Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and  
“Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [7]:
# scraping product url
url=[]
start = 0
end=3
for i in range(start,end): # creating loop to fetch url from 3 pages
    link_tag=driver.find_elements(By.XPATH,'//h2[@class="a-size-mini a-spacing-none a-color-base s-line-clamp-4"]/a')
    for l in link_tag:
        url.append(l.get_attribute('href'))
        
    next=driver.find_element(By.XPATH,'//a[@class="s-pagination-item s-pagination-next s-pagination-button s-pagination-separator"]')
    next.click()
    time.sleep(5)

In [8]:
len(url)

156

In [9]:
# fetching mentioned product details from url
# creating empty lists for product details
brand=[]
name=[]
price=[] 
ereturn=[]
delivery=[] 
availability=[]
for i in url: #iterating the urls stored in url list
    driver.get(i) # taking each url one by one
    time.sleep(20)
    #fetching product details from each url
    try:
        b_tag=driver.find_element(By.XPATH,'//tr[@class="a-spacing-small po-brand"]/td[2]') # product brand name
        brand.append(b_tag.text)
    except NoSuchElementException:
        brand.append('-')

    try:
        name_tag=driver.find_element(By.XPATH,'//span[@id="productTitle"]') # product name
        name.append(name_tag.text)
    except NoSuchElementException:
        name.append('-')

    try:
        price_tag=driver.find_element(By.XPATH,'//span[@class="a-price aok-align-center"]') # product price
        price.append(re.sub('₹|\n00','',price_tag.text))
    except NoSuchElementException:
        price.append('-')

    try:
        ret_tag=driver.find_element(By.XPATH,'//li[@class="a-carousel-card tw-scroll-carousel-element"][3]/div/span/div[2]/span') # return/exchange detail
        ereturn.append(ret_tag.text)
    except NoSuchElementException:
        ereturn.append('-')

    try:
        del_tag=driver.find_element(By.XPATH,'//div[@class="a-spacing-base"][1]/span/span') # expected delivery date of product
        delivery.append(del_tag.text)
    except NoSuchElementException:
        delivery.append('-')

    try:
        avail=driver.find_element(By.ID,"availability") #availability of the product
        availability.append(avail.text)
    except NoSuchElementException:
        availability.append('-')

In [10]:
print(len(brand),len(name),len(price),len(ereturn),len(delivery),len(availability))

156 156 156 156 156 156


In [11]:
#creating dataframe for fetched details
df=pd.DataFrame({"Brand Name":brand,"Name of the Product":name,"Price(₹)":price,"Return/Exchange":ereturn,"Expected Delivery":delivery,"Availability":availability,"Product URL":url})
df

,Brand Name,Name of the Product,Price(₹),Return/Exchange,Expected Delivery,Availability,Product URL
0,Kadence,"Kadence rosewood Guitar Frontier Series, Elect...","5,399",7 days Replacement,"Saturday, 8 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
1,Kadence,Kadence Frontier guitar with Online Guitar lea...,"4,899",7 days Replacement,"Sunday, 9 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
2,Intern,INTERN 41 inches Cutaway Shape Acoustic Guitar...,"3,499",7 days Replacement,7 - 9 June,,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
3,Intern,INTERN 40 inches Acoustic Guitar with Pick-up ...,"4,399",7 days Replacement,"Monday, 10 June",In stock,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...
4,JUAREZ,"Juârez Acoustic Guitar Kit, 38 Inch Cutaway, 0...","2,195",7 days Replacement,"Monday, 10 June",In stock,https://www.amazon.in/Juarez-Acoustic-Cutaway-...
...,...,...,...,...,...,...,...
151,MOCKING BIRD,MOCKING BIRD 38 Inch Acoustic Guitar for Begin...,"3,244",7 days Replacement,"Saturday, 8 June",In stock,https://www.amazon.in/MOCKING-BIRD-Beginners-I...
152,VAULT,Vault RG1 Soloist Premium Electric Guitar - Tr...,"11,999",7 days Replacement,"Saturday, 8 June",In stock,https://www.amazon.in/Soloist-Premium-Electric...
153,Kadence,"Kadence Slowhand Cedar Top SHM02, Premium Semi...","9,499",7 days Replacement,"Sunday, 9 June",In stock,https://www.amazon.in/Kadence-Slowhand-Premium...
154,Aria,Aria FET M2 Meranti Thin-Body Cutaway Acoustic...,"6,879",7 days Replacement,"Sunday, 9 June",Only 1 left in stock.,https://www.amazon.in/Aria-Meranti-Thin-Body-C...


In [12]:
#saving into csv file
df.to_csv('AmazonGuitar.csv')

Q3-- Write a python program to access the search bar and search button on images.google.com and scrape 10 
images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’

In [38]:
driver.get('https://images.google.com/') # opening google image search in automated chrome

In [39]:
input=driver.find_element(By.CLASS_NAME,"gLFyf") #search box element

In [40]:
input.send_keys('Cake') #entering string to search

In [41]:
# searching for results
search=driver.find_element(By.CLASS_NAME,"Tg7LZd")
search.click()

In [42]:
# Scrolling down for more contents
for _ in range(20):
    driver.execute_script("window.scrollBy(0,500)")

# creating empty list for image url
image_url=[]
im_tag=driver.find_elements(By.XPATH,'//div[@class="H8Rx8c"]/g-img/img')
for i in im_tag: # fetching image url
    source=i.get_attribute('src')
    if source is not None: # storing correct image url in the empty list
        if (source[0:4]=='http'):
            image_url.append(source)

In [48]:
len(image_url)

80

In [47]:
#downloading images from stored image urls
for i in range(len(image_url)):
    if i>10:
        break
    print('Downloading {0} of {1} images'.format(i,10))
    response=requests.get(image_url[i])
    file=open(r"C:\Users\anura\OneDrive\Desktop\Fliprorbo"+str(i)+".jpg","wb")
    file.write(response.content)

Q4-- Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.com 
and scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand 
Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”,  
“Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the 
details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [3]:
driver.get('https://www.flipkart.com/') #opening flipkart in automated chrome

In [4]:
input=driver.find_element(By.CLASS_NAME,"Pke_EE") # searchbox element

In [5]:
input.send_keys('Oneplus Nord') #entering product

In [6]:
search=driver.find_element(By.CLASS_NAME,"_2iLD__")
search.click() #search click

In [7]:
# creating empty list for product url
url=[]
link=driver.find_elements(By.XPATH,'//a[@class="CGtC98"]')
for l in link:
    url.append(l.get_attribute('href')) # storing product url

print(len(url))

24


In [8]:
# creating empty list to store mentioned product details
brand=[]
name=[]
colour=[]
RAM=[]
ROM=[]
p_camera=[]
s_camera=[]
display_size=[]
battery_capacity=[]
price=[]

for u in url: # iterating urls from url list

    driver.get(u) # fetching products details from each url
    time.sleep(10)
    # Product Brand
    try: 
        b_tag=driver.find_element(By.CLASS_NAME,"VU-ZEz")
        brand.append(b_tag.text[0:7]) 
    except NoSuchElementException:
        brand.append('-')

     # Product Name
    try: 
        name_tag=driver.find_element(By.CLASS_NAME,"VU-ZEz")
        name.append(name_tag.text) 
    except NoSuchElementException:
        name.append('-')

    # product colour
    try: 
        clr=driver.find_element(By.XPATH,'//div[@class="_1OjC5I pKBIUH"]/div[1]/table/tbody/tr[4]/td[2]/ul/li') 
        colour.append(clr.text)
    except NoSuchElementException:
        colour.append('-')
        
    # product RAM & ROM
    try: 
        ram=driver.find_element(By.XPATH,'//div[@class="xFVion"]/ul/li[1]') 
        ra=re.search('(\d+.+RAM) \|( \d+.+ROM)',ram.text)
        RAM.append(ra.group(1).replace(' RAM',''))
        ROM.append(ra.group(2).replace(' ROM',''))
    except NoSuchElementException:
        RAM.append('-')
        ROM.append('-')
    
    # product primary camera & secondary camera
    try:
        ps=driver.find_element(By.XPATH,'//div[@class="xFVion"]/ul/li[3]') 
        p=re.search('\d+MP *R',ps.text)
        s=re.search('\d+MP *F',ps.text)
        p_camera.append(p.group().replace(' R',''))
        if s is None:
            s_camera.append('-')
        else:
            s_camera.append(s.group().replace(' F',''))
    except NoSuchElementException:
        p_camera.append('-')
        s_camera.append('-')
        
    # product display size
    try:
        display=driver.find_element(By.XPATH,'//div[@class="xFVion"]/ul/li[2]') 
        d=re.search('\d+.+\)',display.text)
        display_size.append(d.group())
    except NoSuchElementException:
        display_size.append('-')
   
    # product battery capacity
    try:
        battery=driver.find_element(By.XPATH,'//div[@class="xFVion"]/ul/li[4]') 
        b=re.search('\d+.+mAh',battery.text)
        battery_capacity.append(b.group())
    except NoSuchElementException:
        battery_capacity.append('-')

    # product price
    try:
        p_tag=driver.find_element(By.XPATH,'//div[@class="x+7QT1"]/div/div/div[1]') 
        price.append(p_tag.text.replace('₹',''))
    except NoSuchElementException:
        price.append('-')

print(len(brand),len(name),len(colour),len(RAM),len(ROM),len(p_camera),len(s_camera),len(display_size),len(battery_capacity),len(price))

24 24 24 24 24 24 24 24 24 24


In [10]:
# Creating datframe for scraped data
df_flipkart=pd.DataFrame({'Brand Name':brand,'Smartphone name':name,'Colour':colour,'RAM':RAM,'Storage(ROM)':ROM,'Primary Camera':p_camera,'Secondary Camera':s_camera,'Display Size':display_size,'Battery Capacity':battery_capacity,'Price(₹)':price,'Product URL':url})
df_flipkart

,Brand Name,Smartphone name,Colour,RAM,Storage(ROM),Primary Camera,Secondary Camera,Display Size,Battery Capacity,Price(₹),Product URL
0,OnePlus,"OnePlus Nord CE 3 Lite 5G (Pastel Lime, 128 GB...",Pastel Lime,8 GB,128 GB,108MP,-,17.07 cm (6.72 inch),5000 mAh,"16,797",https://www.flipkart.com/oneplus-nord-ce-3-lit...
1,OnePlus,"OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 128...",Chromatic Gray,8 GB,128 GB,108MP,-,17.07 cm (6.72 inch),5000 mAh,"16,766",https://www.flipkart.com/oneplus-nord-ce-3-lit...
2,OnePlus,"OnePlus Nord CE 3 Lite 5G (Pastel Lime, 256 GB...",Pastel Lime,8 GB,256 GB,108MP,-,17.07 cm (6.72 inch),5000 mAh,"18,298",https://www.flipkart.com/oneplus-nord-ce-3-lit...
3,OnePlus,"OnePlus Nord CE 3 Lite 5G (Chromatic Gray, 256...",Chromatic Gray,8 GB,256 GB,108MP,-,17.07 cm (6.72 inch),5000 mAh,"18,570",https://www.flipkart.com/oneplus-nord-ce-3-lit...
4,OnePlus,"OnePlus Nord 3 5G (Tempest Gray, 128 GB) (8 G...",Tempest Gray,8 GB,128 GB,50MP,-,17.12 cm (6.74 inch),5000 mAh,"23,499",https://www.flipkart.com/oneplus-nord-3-5g-tem...
5,OnePlus,"OnePlus Nord 3 5G (Misty Green, 256 GB) (16 G...",Misty Green,16 GB,256 GB,50MP,-,17.12 cm (6.74 inch),5000 mAh,"37,630",https://www.flipkart.com/oneplus-nord-3-5g-mis...
6,OnePlus,"OnePlus Nord 3 5G (Misty Green, 128 GB) (8 GB...",Misty Green,8 GB,128 GB,50MP,-,17.12 cm (6.74 inch),5000 mAh,"21,703",https://www.flipkart.com/oneplus-nord-3-5g-mis...
7,OnePlus,"OnePlus Nord 3 5G (Tempest Gray, 256 GB) (16 ...",Tempest Gray,16 GB,256 GB,50MP,-,17.12 cm (6.74 inch),5000 mAh,"36,000",https://www.flipkart.com/oneplus-nord-3-5g-tem...
8,OnePlus,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB) ...",Blue Tide,6 GB,128 GB,64MP,16MP,16.74 cm (6.59 inch),5000 mAh,"16,197",https://www.flipkart.com/oneplus-nord-ce-2-lit...
9,OnePlus,"OnePlus Nord CE4 (Dark Chrome, 128 GB) (8 GB ...",Dark Chrome,8 GB,128 GB,50MP,-,17.02 cm (6.7 inch),5500 mAh,"24,686",https://www.flipkart.com/oneplus-nord-ce4-dark...


In [11]:
#saving data into csv file
df_flipkart.to_csv('flipkart_oneplus.csv')

Q5-- Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps.

In [10]:
# opening google map
driver.get('https://www.google.co.in/maps/@25.608192,85.11488,12z?entry=ttu')

In [11]:
#searching for a city on google maps
city=driver.find_element(By.XPATH,'//input[@class="fontBodyMedium searchboxinput xiQnY "]')

In [12]:
city.send_keys('Edinburgh')

In [13]:
search=driver.find_element(By.ID,"searchbox-searchbutton")
search.click()

In [14]:
# extracting latitude and longitude from city url
try:
    city_url=driver.current_url
    print('URL Extracted- ',city_url)
    lat_long=re.findall(r'@.+data',city_url)
    print(lat_long)
except:
    print('nothing found')

URL Extracted-  https://www.google.co.in/maps/place/Edinburgh,+UK/@55.9410984,-3.3701468,11z/data=!3m1!4b1!4m6!3m5!1s0x4887b800a5982623:0x64f2147b7ce71727!8m2!3d55.953252!4d-3.188267!16zL20vMDJtNzc?entry=ttu
['@55.9410984,-3.3701468,11z/data']


In [22]:
# Extracting Latitude and longitude seperately
l=re.findall(r'-?\d+\.\d+\d\b',str(lat_long))
l

['55.9410984', '-3.3701468']

In [23]:
# Storing data
city='Edinburgh'
latitude=[]
longitude=[]
latitude.append(l[0])
longitude.append(l[1])

In [24]:
#Creating dataframe for city with it's latitude and longitude
df=pd.DataFrame({'City':city,'Latitude':latitude,'Longitude':longitude})
df

,City,Latitude,Longitude
0,Edinburgh,55.9410984,-3.3701468


Q6-- Write a program to scrap all the available details of best ai laptops from digit.in.

In [71]:
#opening digit.in
driver.get('https://www.digit.in/')

In [72]:
#selecting top 10 products section
top_10=driver.find_element(By.XPATH,'//ul[@class="menu off-canvas off-canvas-active"]/li[4]')
top_10.click()

In [73]:
#searching ai laptops
ai=driver.find_element(By.XPATH,'//div[@class="news-community clearfix"][4]/div/p[1]')
ai.click()

In [84]:
# Creating empty list to store product details
name=[]
o_system=[]
d_size=[]
processor=[]
# product name
try:
    n_tag=driver.find_elements(By.XPATH,'//h3[@class="font130 mt0 mb10 mobilesblockdisplay "]')
    for n in n_tag:
        name.append(n.text)
except NoSuchElementException:
    print('NoSuchElement')

# operating system
try:
    o_tag=driver.find_elements(By.XPATH,'//div[@class="woo_code_zone_loop clearbox"]/div[1]/div/span[2]')
    for o in o_tag:
        o_system.append(o.text)
except NoSuchElementException:
    print('NoSuchElement')

# Display size
try:
    display_tag=driver.find_elements(By.XPATH,'//div[@class="woo_code_zone_loop clearbox"]/div[2]/div/span[2]')
    for d in display_tag:
        d_size.append(d.text)
except NoSuchElementException:
    print('NoSuchElement')

# Processor
try:
    p_tag=driver.find_elements(By.XPATH,'//div[@class="woo_code_zone_loop clearbox"]/div[4]/div/span[2]')
    for p in p_tag:
        processor.append(p.text)
except NoSuchElementException:
    print('NoSuchElement')

print(len(name),len(o_system),len(d_size),len(processor))

8 8 8 8


In [85]:
# Creating dataframe for scraped data
ai_laptop=pd.DataFrame({'Name':name,'Operating System':o_system,'Display Size':d_size,'Processor':processor})
ai_laptop

,Name,Operating System,Display Size,Processor
0,MSI Prestige 16 AI Evo B1M,Windows 11 Home,16,Intel Core Ultra 9
1,Asus ROG Zephyrus G14 GA402XZ-N2019WS,Windows 11 Home,14,"7th Gen, AMD Ryzen 9 7940HS"
2,Acer Predator Triton Neo 16,Windows 11 Home,16,Intel Core Ultra 9 185H
3,ASUS Zenbook 14 OLED (UX3405),Windows 11 Pro,14,Intel Core Ultra 7-155H
4,HP Spectre x360 16-aa0664TX,Windows 11 Home,16,Intel Core Ultra 7-155H
5,Lenovo IdeaPad Pro 5 Core Ultra,Windows 11 Home,16,Intel Core Ultra 7-155H
6,Acer Nitro V 16 (ANV16-41),Windows 11 Home,16,AMD Ryzen 7-7840HS
7,Acer Swift Go 14 SFG14-72T,Windows 11 Home,14,Intel Core Ultra 7-155H


Q7-- Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: 
“Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [78]:
#opening forbes billionaire list
driver.get('https://www.forbes.com/billionaires/')

In [122]:
# Creating Empty list for scraped data
rank=[]
name=[]
worth=[]
age=[]
citizen=[]
source=[]
industry=[]
#Rank Of Billionaires
r_tag=driver.find_elements(By.XPATH,'//div[@class="Table_tableRow__lF_cY"]/div[1]')
for r in r_tag:
    rank.append(r.text.replace('.',''))

# Name of Billionaires
n_tag=driver.find_elements(By.XPATH,'//div[@class="Table_tableRow__lF_cY"]/div[2]')
for n in n_tag:
    name.append(n.text)

# Worth of Billionaires
w_tag=driver.find_elements(By.XPATH,'//div[@class="Table_tableRow__lF_cY"]/div[3]')
for w in w_tag:
    worth.append(w.text)

# Billionaires industries
age_tag=driver.find_elements(By.XPATH,'//div[@class="Table_tableRow__lF_cY"]/div[4]')
for a in age_tag:
    age.append(a.text)

# Billionaires citizenship
city=driver.find_elements(By.XPATH,'//div[@class="Table_tableRow__lF_cY"]/div[5]')
for c in city:
    citizen.append(c.text)

# Billionaires source
source_tag=driver.find_elements(By.XPATH,'//div[@class="Table_tableRow__lF_cY"]/div[6]')
for s in source_tag:
    source.append(s.text)

# Billionaires industry
ind_tag=driver.find_elements(By.XPATH,'//div[@class="Table_tableRow__lF_cY"]/div[7]')
for i in ind_tag:
    industry.append(i.text)

print(len(rank),len(name),len(worth),len(age),len(citizen),len(source),len(industry))

200 200 200 200 200 200 200


In [123]:
#creating dataframe
df=pd.DataFrame({'Rank':rank,'Name':name,'Worth':worth,'Age':age,'Citizenship':citizen,'Source':source,'Industry':industry})
df

,Rank,Name,Worth,Age,Citizenship,Source,Industry
0,1,Bernard Arnault & family,$233 B,75,France,LVMH,Fashion & Retail
1,2,Elon Musk,$195 B,52,United States,"Tesla, SpaceX",Automotive
2,3,Jeff Bezos,$194 B,60,United States,Amazon,Technology
3,4,Mark Zuckerberg,$177 B,39,United States,Facebook,Technology
4,5,Larry Ellison,$141 B,79,United States,Oracle,Technology
...,...,...,...,...,...,...,...
195,195,Lei Jun,$10.9 B,54,China,Smartphones,Technology
196,195,Georg Schaeffler,$10.9 B,59,Germany,Auto parts,Automotive
197,195,Marcel Herrmann Telles & family,$10.9 B,74,Brazil,Beer,Food & Beverage
198,199,David Velez & family,$10.8 B,42,Colombia,Fintech,Finance & Investments


In [124]:
# saving Billionaires dataframe into csv file 
df.to_csv('billionaires.csv')

Q8-- Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted 
from any YouTube Video

In [201]:
# opening Joker: Folie à Deux teaser trailer on youtube on automated chrome
driver.get('https://www.youtube.com/watch?v=xy8aJw1vYHo')

In [211]:
#creating empty list to store scraped data
comment=[]
times=[]
upvote=[]
for _ in range(25): #scrolling 
    driver.execute_script("window.scrollBy(0,500)")

    #Comment on the video
    try:
        com_tag=driver.find_elements(By.XPATH,'//div[@class="style-scope ytd-expander"]/yt-attributed-string')
        for c in com_tag:
            comment.append(c.text)
    except NoSuchElementException:
        comment.append('-')

# comment time
    try:
        time_tag=driver.find_elements(By.XPATH,'//span[@id="published-time-text"]/a')
        for t in time_tag:
            times.append(t.text)
    except NoSuchElementExecption:
        times.append('-')

#comment upvote
    try:
        vote_tag=driver.find_elements(By.XPATH,'//span[@id="vote-count-middle"]')
        for v in vote_tag:
            upvote.append(v.text)
    except NoSuchElementException:
        upvote.append('-')



print(len(comment),len(times),len(upvote))

4940 5020 5020


In [212]:
df=pd.DataFrame({"Comment":comment[0:500],"Comment Time":times[0:500],"Comment Upvote":upvote[0:500]})
df

,Comment,Comment Time,Comment Upvote
0,Who ever came up with the idea of glass lipsti...,1 month ago,62K
1,"Wouldn't even say this is a teaser, this is ex...",1 month ago,61K
2,This is pure cinema. The shots look incredible...,2 weeks ago,310
3,This is the first Joker actor in three decades...,4 weeks ago,1.1K
4,That Lipstick on the glass at the end was just...,1 month ago,56K
...,...,...,...
495,Joaquin and Gaga are pure magic together,2 weeks ago,21
496,That last shot was cold asf.,1 month ago,10K
497,I don't need a trailer. This teaser was perfec...,2 weeks ago,51
498,I smell a award brewing,1 month ago,23


Q9 -- Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in 
“London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall 
reviews, privates from price, dorms from price, facilities and property description.

In [3]:
# opening hostelworld for london location
driver.get('https://www.hostelworld.com/pwa/wds/s?q=London,%20England&country=England&city=London&type=city&id=3&from=2024-06-05&to=2024-06-08&guests=1&page=1')

In [4]:
# Scraping details of hostel in london
hostel=[]
distance=[]
ratings=[]
total_reviews=[]
overall_reviews=[]
privates=[]
dorms=[]

# hostel name
name_tag=driver.find_elements(By.XPATH,'//div[@class="property-name"]/span')
for n in name_tag:
    hostel.append(n.text)

#Distance from city
dis_tag=driver.find_elements(By.XPATH,'//span[@class="distance-description"]')
for d in dis_tag:
    distance.append(re.sub('- | from city centre','',d.text))

# Ratings of hostel
rating_tag=driver.find_elements(By.XPATH,'//div[@class="rating medium theme-null"]/div[1]/span[1]')
for r in rating_tag:
    ratings.append(r.text)

#Total review of hostel
t_review=driver.find_elements(By.XPATH,'//div[@class="rating medium theme-null"]/div[2]/span')
for t in t_review:
    total_reviews.append(re.sub('[()]','',t.text))

#overall review of hostel
o_review=driver.find_elements(By.XPATH,'//div[@class="rating medium theme-null"]/div[1]/span[2]')
for o in o_review:
    overall_reviews.append(o.text)

#Privates from prices of hostel
private_tag=driver.find_elements(By.XPATH,'//div[@class="property-accommodation-prices"]/div[1]')
for p in private_tag:
    privates.append(re.sub('-\d+%|Privates From','',p.text).replace('\n',' '))

# Dorms from prices of hostel
dorms_tag=driver.find_elements(By.XPATH,'//div[@class="property-accommodation-prices"]/div[2]')
for d in dorms_tag:
    dorms.append(re.sub('-\d+%|Dorms From','',d.text).replace('\n',' '))
print(len(hostel),len(distance),len(ratings),len(total_reviews),len(overall_reviews),len(privates),len(dorms))

30 30 30 30 30 30 30


In [5]:
# Scraping Hostel url
url=[]
link=driver.find_elements(By.XPATH,'//a[@class="property-card-container horizontal"]')
for l in link:
    url.append(l.get_attribute('href'))

print(len(url))

30


In [6]:
# scraping further hostel details iterating hostel url
facilities=[]
property_desc=[]
for u in url:
    driver.get(u)
    time.sleep(5)

    # Hostel Facilities
    try:
        facility=driver.find_element(By.XPATH,'//div[@class="facilities snippet"]')
        facilities.append(facility.text.replace('\n',','))
    except NoSuchElementException:
        facilities.append('-')

    # Hostel Description
    try:
        property=driver.find_element(By.XPATH,'//div[@class="about-description"]/p')
        property_desc.append(re.sub('\n+\*?','',property.text))
    except NoSuchElementException:
        property_desc.append('-')

print(len(facilities),len(property_desc))

30 30


In [9]:
#Creating Dataframe of scraped hostel data
df=pd.DataFrame({'Hostel name':hostel,'Distance from city centre':distance,'Ratings':ratings,'Total reviews':total_reviews,'Overall reviews':overall_reviews,'Privates from price':privates,'Dorms from price':dorms,'Facilities':facilities,'Property description':property_desc})
df

,Hostel name,Distance from city centre,Ratings,Total reviews,Overall reviews,Privates from price,Dorms from price,Facilities,Property description
0,Wombat's City Hostel London,3.6km,9.3,16481,Superb,"₹20,292 ₹16,233","₹4,937 ₹3,949","Linen Included,Free City Maps,Free WiFi,Free I...",A safe haven in the middle of the metropolis: ...
1,Palmers Lodge - Swiss Cottage,6.5km,8.7,16202,Fabulous,No Privates Available,"₹3,869 ₹2,863","Linen Included,Free WiFi,Free Internet Access,...","Whether you're a backpacker, flash-packer or a..."
2,Onefam Waterloo,0.7km,9.5,346,Superb,"₹11,407","₹4,024","Linen Included,Free WiFi,Free Internet Access,...",Come join us at our newest Onefam location in ...
3,NX London Hostel,6.1km,8.8,2370,Fabulous,No Privates Available,"₹4,112 ₹3,495","Free Breakfast,Linen Included,Towels Included,...",Welcome to NX London Hostel!PLEASE READ INFORM...
4,Urbany Hostel London,5.4km,9.6,1278,Superb,No Privates Available,"₹3,802","Linen Included,Free WiFi,Security Lockers,Micr...",Welcome to Urbany Hostel London! Ideal for the...
5,Onefam Notting Hill,5.5km,9.7,2820,Superb,"₹14,159","₹4,275","Linen Included,Free WiFi,Free Internet Access,...",The perfect place for solo travelers to connec...
6,St Christopher's Village,1.8km,7.5,13079,Very Good,No Privates Available,"₹2,861 ₹2,575","Linen Included,Free City Maps,Free WiFi,Free I...",St Christopher's Inn at The Village in London ...
7,Generator London,3km,7.2,8262,Very Good,"₹10,350","₹3,066","Linen Included,Free City Maps,Free WiFi,Free I...",Generator London is a design hotel-hostel loca...
8,No.8 Seven Sisters,9km,7.8,4278,Very Good,No Privates Available,"₹1,805","Linen Included,Free WiFi,Security Lockers,Key ...","For Great Facilities, Comfort & Affordability ..."
9,Astor Museum Inn,2.2km,8.6,9530,Fabulous,No Privates Available,"₹3,074 ₹2,767","Linen Included,Free City Maps,Free WiFi,Securi...",With the British Museum literally on its doors...
